In [ ]:
from graph_tool.all import *
import graph_tool.all as gt
import numpy as np
import scipy as sp
from scipy.integrate import simpson
import random
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from PyGnuplot import gp
import subprocess

In [ ]:
Node = 10000

m = 2

times = 100

M_list  = [5, 10, 20, 50, 100, 200]

beta_list = [1,1/2,0,-1,-3,-5,-100]

net_list = ["SF", "Cutoff", "RA", "IPA_beta1", "IPA_beta3", "IPA_beta5", "IPA_beta100"]

In [ ]:
from decimal import Decimal

w_list = [0.00, 0.50, 0.70, 0.90, 0.95, 0.98]

w_list = [Decimal(str(item)).quantize(Decimal('0.00')) for item in w_list]

In [ ]:
for M in tqdm(M_list):
    all_q_list = []
    for net in tqdm(net_list):
        q_net_list = []
        for w in tqdm(w_list):
            q = []
            for i in range(times):
                g = load_graph(f"~/o_t_hayashilab/Network_data/graph-tool/modular_structure/{net}/w={w}/m0={M}/{i}.gt.gz")
                
                state = gt.BlockState(g, b=g.vp.module_number)
                Q = gt.modularity(g, state.get_blocks())
                q.append(Q)
            
            Q_avg = np.mean(q)
            q_net_list.append(Q_avg)
            
        all_q_list.append(q_net_list)
        
    # 保存先のディレクトリが存在しない場合は作成
    save_dir = f'~/o_t_hayashilab/modular_network_info/N={Node}/m0={M}/'
    os.makedirs(os.path.expanduser(save_dir), exist_ok=True)
    np.save(os.path.join(os.path.expanduser(save_dir), 'modularity_q.npy'), np.array(all_q_list))

In [ ]:
def plot_correlation(x_values, y_values, label, color, marker='o', size=100, linestyle='-'):  # linestyle引数を追加
    plt.scatter(x_values, y_values, label=label, color=color, marker=marker, s=size)  # 散布図を作成
    plt.plot(x_values, y_values, color=color, linestyle=linestyle)  # 線を引く

for M in M_list:
    save_path = f"~/o_t_hayashilab/modular_network_info/N={Node}/m0={M}/ID_R_list.npy"
    ID_R_file = np.load(os.path.expanduser(save_path), allow_pickle=True)
    save_path = f"~/o_t_hayashilab/modular_network_info/N={Node}/m0={M}/modularity_q.npy"
    q_file = np.load(os.path.expanduser(save_path), allow_pickle=True)
    
    # 散布図を作成
    plt.figure(figsize=(8, 6))
    
    plot_correlation(q_file[0], ID_R_file[0], label='v=1', color='red', size=75)
    
    plot_correlation(q_file[4], ID_R_file[4], label='v=-3', color='blue', size=75)
    
    plot_correlation(q_file[1], ID_R_file[1], label='v=1/2', color='orange', size=75)
    
    plot_correlation(q_file[5], ID_R_file[5], label='v=-5', color='cyan', size=75)
    
    plot_correlation(q_file[2], ID_R_file[2], label='v=0', color='yellow', size=75)
    
    plot_correlation(q_file[6], ID_R_file[6], label='v=-100', color='purple', size=75)
    
    plot_correlation(q_file[3], ID_R_file[3], label='v=-1', color='green', size=75)
    
    # 軸ラベルの設定
    plt.xlabel(r"$Q$", fontsize=20)
    plt.ylabel(r"$R^{ID}$", fontsize=20)
    
    # x軸の範囲を設定（0から最大値+0.05）
    plt.xlim(0, 1.0)
    # x軸の目盛りを調整（ここでは0.1刻みに設定し、フォントサイズを15に）
    plt.xticks(ticks=[0, 0.2, 0.4, 0.6, 0.8, 1.0], fontsize=15)
    
    # y軸の範囲を設定（0から最大値+0.05）
    plt.ylim(0, 0.5)
    plt.yticks(ticks=[0, 0.1, 0.2, 0.3, 0.4, 0.5], fontsize=15)
    
    # 凡例の設定（上に2×2で配置）
    plt.legend(loc='upper center', fontsize=16, bbox_to_anchor=(0.5, 1.19), ncol=4, handletextpad=0.4)
    
    # 保存先のディレクトリを指定
    save_dir = os.path.expanduser('~/o_t_hayashilab/modular_network_info/q_R_comparison/')
    os.makedirs(save_dir, exist_ok=True)  # ディレクトリが存在しない場合は作成
    # EPS形式で図を保存（フルパスを指定）
    save_path = os.path.join(save_dir, f"R^ID_mo{M}.eps")
    plt.savefig(save_path, format='eps', bbox_inches='tight')
    
    # プロットの表示
    plt.show()

In [ ]:
def plot_correlation(x_values, y_values, label, color, marker='o', size=100, linestyle='-'):  # linestyle引数を追加
    plt.scatter(x_values, y_values, label=label, color=color, marker=marker, s=size)  # 散布図を作成
    plt.plot(x_values, y_values, color=color, linestyle=linestyle)  # 線を引く

for M in M_list:
    save_path = f"~/o_t_hayashilab/modular_network_info/N={Node}/m0={M}/IB_R_list.npy"
    IB_R_file = np.load(os.path.expanduser(save_path), allow_pickle=True)
    save_path = f"~/o_t_hayashilab/modular_network_info/N={Node}/m0={M}/modularity_q.npy"
    q_file = np.load(os.path.expanduser(save_path), allow_pickle=True)
    
    # 散布図を作成
    plt.figure(figsize=(8, 6))
    
    plot_correlation(q_file[0], IB_R_file[0], label='v=1', color='red', size=75)
    
    plot_correlation(q_file[4], IB_R_file[4], label='v=-3', color='blue', size=75)
    
    plot_correlation(q_file[1], IB_R_file[1], label='v=1/2', color='orange', size=75)
    
    plot_correlation(q_file[5], IB_R_file[5], label='v=-5', color='cyan', size=75)
    
    plot_correlation(q_file[2], IB_R_file[2], label='v=0', color='yellow', size=75)
    
    plot_correlation(q_file[6], IB_R_file[6], label='v=-100', color='purple', size=75)
    
    plot_correlation(q_file[3], IB_R_file[3], label='v=-1', color='green', size=75)
    
    # 軸ラベルの設定
    plt.xlabel(r"$Q$", fontsize=20)
    plt.ylabel(r"$R^{IB}$", fontsize=20)
    
    # x軸の範囲を設定（0から最大値+0.05）
    plt.xlim(0, 1.0)
    # x軸の目盛りを調整（ここでは0.1刻みに設定し、フォントサイズを15に）
    plt.xticks(ticks=[0, 0.2, 0.4, 0.6, 0.8, 1.0], fontsize=15)
    
    # y軸の範囲を設定（0から最大値+0.05）
    plt.ylim(0, 0.5)
    plt.yticks(ticks=[0, 0.1, 0.2, 0.3, 0.4, 0.5], fontsize=15)
    
    # 凡例の設定（上に2×2で配置）
    plt.legend(loc='upper center', fontsize=16, bbox_to_anchor=(0.5, 1.19), ncol=4, handletextpad=0.4)
    
    # 保存先のディレクトリを指定
    save_dir = os.path.expanduser('~/o_t_hayashilab/modular_network_info/q_R_comparison/')
    os.makedirs(save_dir, exist_ok=True)  # ディレクトリが存在しない場合は作成
    # EPS形式で図を保存（フルパスを指定）
    save_path = os.path.join(save_dir, f"R^IB_mo{M}.eps")
    plt.savefig(save_path, format='eps', bbox_inches='tight')
    
    # プロットの表示
    plt.show()

In [ ]:
def plot_correlation(x_values, y_values, label, color, marker='o', size=100, linestyle='-'):  # linestyle引数を追加
    plt.scatter(x_values, y_values, label=label, color=color, marker=marker, s=size)  # 散布図を作成
    plt.plot(x_values, y_values, color=color, linestyle=linestyle)  # 線を引く

for M in M_list:
    save_path = f"~/o_t_hayashilab/modular_network_info/N={Node}/m0={M}/MB_R_list.npy"
    MB_R_file = np.load(os.path.expanduser(save_path), allow_pickle=True)
    save_path = f"~/o_t_hayashilab/modular_network_info/N={Node}/m0={M}/modularity_q.npy"
    q_file = np.load(os.path.expanduser(save_path), allow_pickle=True)
    
    # 散布図を作成
    plt.figure(figsize=(8, 6))
    
    plot_correlation(q_file[0], MB_R_file[0], label='v=1', color='red', size=75)
    
    plot_correlation(q_file[4], MB_R_file[4], label='v=-3', color='blue', size=75)
    
    plot_correlation(q_file[1], MB_R_file[1], label='v=1/2', color='orange', size=75)
    
    plot_correlation(q_file[5], MB_R_file[5], label='v=-5', color='cyan', size=75)
    
    plot_correlation(q_file[2], MB_R_file[2], label='v=0', color='yellow', size=75)
    
    plot_correlation(q_file[6], MB_R_file[6], label='v=-100', color='purple', size=75)
    
    plot_correlation(q_file[3], MB_R_file[3], label='v=-1', color='green', size=75)
    
    # 軸ラベルの設定
    plt.xlabel(r"$Q$", fontsize=20)
    plt.ylabel(r"$R^{MB}$", fontsize=20)
    
    # x軸の範囲を設定（0から最大値+0.05）
    plt.xlim(0, 1.0)
    # x軸の目盛りを調整（ここでは0.1刻みに設定し、フォントサイズを15に）
    plt.xticks(ticks=[0, 0.2, 0.4, 0.6, 0.8, 1.0], fontsize=15)
    
    # y軸の範囲を設定（0から最大値+0.05）
    plt.ylim(0, 0.5)
    plt.yticks(ticks=[0, 0.1, 0.2, 0.3, 0.4, 0.5], fontsize=15)
    
    # 凡例の設定（上に2×2で配置）
    plt.legend(loc='upper center', fontsize=16, bbox_to_anchor=(0.5, 1.19), ncol=4, handletextpad=0.4)
    
    # 保存先のディレクトリを指定
    save_dir = os.path.expanduser('~/o_t_hayashilab/modular_network_info/q_R_comparison/')
    os.makedirs(save_dir, exist_ok=True)  # ディレクトリが存在しない場合は作成
    # EPS形式で図を保存（フルパスを指定）
    save_path = os.path.join(save_dir, f"R^MB_mo{M}.eps")
    plt.savefig(save_path, format='eps', bbox_inches='tight')
    
    # プロットの表示
    plt.show()

In [ ]:
def plot_correlation(x_values, y_values, label, color, marker='o', size=100, linestyle='-'):  # linestyle引数を追加
    plt.scatter(x_values, y_values, label=label, color=color, marker=marker, s=size)  # 散布図を作成
    plt.plot(x_values, y_values, color=color, linestyle=linestyle)  # 線を引く

for M in M_list:
    save_path = f"~/o_t_hayashilab/modular_network_info/N={Node}/m0={M}/second_eigenvalue_list.npy"
    μ_file = np.load(os.path.expanduser(save_path), allow_pickle=True)
    save_path = f"~/o_t_hayashilab/modular_network_info/N={Node}/m0={M}/modularity_q.npy"
    q_file = np.load(os.path.expanduser(save_path), allow_pickle=True)
    
    # 散布図を作成
    plt.figure(figsize=(8, 6))
    
    plot_correlation(q_file[0], μ_file[0], label='v=1', color='red', size=75)
    
    plot_correlation(q_file[4], μ_file[4], label='v=-3', color='blue', size=75)
    
    plot_correlation(q_file[1], μ_file[1], label='v=1/2', color='orange', size=75)
    
    plot_correlation(q_file[5], μ_file[5], label='v=-5', color='cyan', size=75)
    
    plot_correlation(q_file[2], μ_file[2], label='v=0', color='yellow', size=75)
    
    plot_correlation(q_file[6], μ_file[6], label='v=-100', color='purple', size=75)
    
    plot_correlation(q_file[3], μ_file[3], label='v=-1', color='green', size=75)
    
    # 軸ラベルの設定
    plt.xlabel(r"$Q$", fontsize=20)
    plt.ylabel(r"$μ_{2}$", fontsize=20)
    
    # x軸の範囲を設定（0から最大値+0.05）
    plt.xlim(0, 1.0)
    # x軸の目盛りを調整（ここでは0.1刻みに設定し、フォントサイズを15に）
    plt.xticks(ticks=[0, 0.2, 0.4, 0.6, 0.8, 1.0], fontsize=15)
    
    # y軸の範囲を設定（0から最大値+0.05）
    plt.ylim(0, 0.6)
    plt.yticks(ticks=[0, 0.1, 0.2, 0.3, 0.4, 0.5], fontsize=15)
    
    # 凡例の設定（上に2×2で配置）
    plt.legend(loc='upper center', fontsize=16, bbox_to_anchor=(0.5, 1.19), ncol=4, handletextpad=0.4)
    
    # 保存先のディレクトリを指定
    save_dir = os.path.expanduser('~/o_t_hayashilab/modular_network_info/q_μ_comparison/')
    os.makedirs(save_dir, exist_ok=True)  # ディレクトリが存在しない場合は作成
    # EPS形式で図を保存（フルパスを指定）
    save_path = os.path.join(save_dir, f"mu2_mo{M}.eps")
    plt.savefig(save_path, format='eps', bbox_inches='tight')
    
    # プロットの表示
    plt.show()